<a href="https://colab.research.google.com/github/khushbu-shah2928/khushbu-shah2928/blob/main/Webbrains_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:

pip install langchain


In [44]:
pip install langgraph

In [45]:
pip install langchain_community

In [51]:
import os
import json
import random
from transformers import pipeline
from langchain.memory import ConversationBufferMemory
from typing import TypedDict
from langgraph.graph import StateGraph

# Load Hugging Face model (using text-generation pipeline)
llm = pipeline("text-generation", model="gpt2")

# Define state using TypedDict
class TravelPlannerState(TypedDict):
    preferences: dict
    destination: str
    itinerary: str

# Load mock destination database
def load_destinations(file_path):
    try:
        with open(file_path, "r") as file:
            return json.load(file)
    except Exception as e:
        print(f"Error loading destinations file: {e}")
        return []

destinations_db = load_destinations("destinations.json")  # Adjust the file path if needed

# Function to validate input
def get_valid_input(prompt, valid_options):
    while True:
        user_input = input(prompt).strip().lower()
        if user_input in valid_options:
            return user_input
        else:
            print(f"Invalid input! Please choose from {', '.join(valid_options)}.")

# Step 1: Extract user preferences
def extract_preferences(state: TravelPlannerState):
    print("Welcome! Let's plan your trip.")
    budget = get_valid_input("What's your budget? (Low/Medium/High): ", ["low", "medium", "high"])
    interests = get_valid_input("What are your interests? (Adventure/Relaxation/Culture/Food): ",
                                ["adventure", "relaxation", "culture", "food"])
    # climate = get_valid_input("Do you prefer warm or cold destinations?: ", ["warm", "cold"])
    season = get_valid_input("Which season are you planning to travel? (Spring/Summer/Fall/Winter): ",
                             ["spring", "summer", "fall", "winter"])

    state["preferences"] = {
        "budget": budget,
        "interests": interests,
        # "climate": climate,
        "season": season
    }

    print("Preferences captured:", state["preferences"])
    return {"preferences": state["preferences"]}  # Return updated state

# Step 2: Find suitable destinations
def find_destinations(state: TravelPlannerState):
    preferences = state["preferences"]

    # Debug print for preferences
    print("Matching preferences:", preferences)

    # Filter destinations based on preferences
    matching_destinations = [
        d for d in destinations_db if
        d.get('budget_level') == preferences['budget'] and
        preferences['interests'] in d.get('tags', []) and
        preferences['season'] in d.get('best_seasons', [])]

    # Debug print for matching destinations
    print("Matching destinations:", matching_destinations)

    if not matching_destinations:
        destination = "No matching destination found. Try adjusting your preferences."
    else:
        destination = random.choice(matching_destinations)['name']

    state["destination"] = destination
    print(f"Based on your preferences, we suggest: {state['destination']}")

    return {"destination": state["destination"]}  # Return updated state

# Step 3: Generate itinerary
def create_itinerary(state: TravelPlannerState):
    destination = state["destination"]
    if "No matching destination found" in destination:
        itinerary = "No itinerary available. Please try adjusting your preferences."
    else:
        itinerary = f"Day 1: Explore {destination}'s famous attractions.\nDay 2: Enjoy local cuisine.\nDay 3: Relax and shop."

    state["itinerary"] = itinerary
    print(f"Here is your itinerary:\n{itinerary}")

    return {"itinerary": state["itinerary"]}  # Return updated state

# Step 4: Handle follow-up questions
def handle_followup(state: TravelPlannerState):
    # Ensure get_valid_input function exists and handles input properly
    question = get_valid_input("Do you have any questions about your trip? (Yes/No): ", ["yes", "no"]).lower()

    if question == "yes":
        user_query = input("Ask your question: ")

        if not user_query.strip():  # Check if input is empty
            print("You didn't ask a question. Please try again.")
            return {}

        # Call Hugging Face's text-generation pipeline for response
        response = llm(user_query, num_return_sequences=10)
        print(f"AI Response:\n {response[0]['generated_text']}")

    return {}  # Return the updated state instead of an empty dictionary

# Build the LangGraph workflow
graph = StateGraph(TravelPlannerState)
graph.add_node("extract_preferences", extract_preferences)
graph.add_node("find_destinations", find_destinations)
graph.add_node("create_itinerary", create_itinerary)
graph.add_node("handle_followup", handle_followup)

# Define workflow transitions
graph.add_edge("extract_preferences", "find_destinations")
graph.add_edge("find_destinations", "create_itinerary")
graph.add_edge("create_itinerary", "handle_followup")

graph.set_entry_point("extract_preferences")

# Initialize state
template_state = TravelPlannerState(
    preferences={},
    destination="",
    itinerary=""
)


# Run the workflow four times
for i in range(4):
    print(f"\n--- Run {i + 1} ---")
    executor = graph.compile()
    executor.invoke(template_state)


Device set to use cpu



--- Run 1 ---
Welcome! Let's plan your trip.
What's your budget? (Low/Medium/High): medium
What are your interests? (Adventure/Relaxation/Culture/Food): culture
Which season are you planning to travel? (Spring/Summer/Fall/Winter): summer
Preferences captured: {'budget': 'medium', 'interests': 'culture', 'season': 'summer'}
Matching preferences: {'budget': 'medium', 'interests': 'culture', 'season': 'summer'}
Matching destinations: [{'name': 'Bali', 'country': 'Indonesia', 'tags': ['adventure', 'relaxation', 'culture'], 'budget_level': 'medium', 'ideal_duration': [5, 10], 'best_seasons': ['summer', 'fall']}]
Based on your preferences, we suggest: Bali
Here is your itinerary:
Day 1: Explore Bali's famous attractions.
Day 2: Enjoy local cuisine.
Day 3: Relax and shop.
Do you have any questions about your trip? (Yes/No): no

--- Run 2 ---
Welcome! Let's plan your trip.
What's your budget? (Low/Medium/High): high
What are your interests? (Adventure/Relaxation/Culture/Food): food
Which seas

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


AI Response:
 What is the budget for paris from May 2015?

This is for the most part a discretionary spending budget. Paris is the budget for the entire country and all the services and expenditures included in it is for this purpose only. The public

--- Run 3 ---
Welcome! Let's plan your trip.
What's your budget? (Low/Medium/High): high
What are your interests? (Adventure/Relaxation/Culture/Food): adventure
Which season are you planning to travel? (Spring/Summer/Fall/Winter): winter
Preferences captured: {'budget': 'high', 'interests': 'adventure', 'season': 'winter'}
Matching preferences: {'budget': 'high', 'interests': 'adventure', 'season': 'winter'}
Matching destinations: [{'name': 'Reykjavik', 'country': 'Iceland', 'tags': ['nature', 'adventure', 'cold'], 'budget_level': 'high', 'ideal_duration': [4, 8], 'best_seasons': ['winter', 'autumn']}]
Based on your preferences, we suggest: Reykjavik
Here is your itinerary:
Day 1: Explore Reykjavik's famous attractions.
Day 2: Enjoy local

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


AI Response:
 where is tokyo-warm water-soluble (SOA) water, with more or less the same amount of dissolved inorganic nitrogen. The composition is generally in terms of pH, which is a very easy way to determine how much SO
